# IR Project 4
## Topic modeling with LDA (Latent Dirichlet Allocation)

### Load libraries and data

In [54]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 30.2 MB/s 
     |████████████████████████████████| 11.3 MB 29.1 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=ef609e59105bbbff501d28a04c8ac34dee8ad03096e1f7cf9c062958347fff45
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take in

In [5]:
# Import libraries
import os
import pickle
import re
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/IRProject")
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['tweet_TR.json',
 'preprocessed_tweets.pickle',
 'tweets_emoSent2.json',
 'TopicModeling (1).ipynb',
 'topicsModel',
 'tweets_emoSentTop.json',
 'tweets_emoSentTopic.json',
 'tweets_emoSentTopics.json',
 'tweets_emoSentTopics2.json',
 'tweets_8Topics.json',
 'words_per_topic.pickle',
 'words_per_topic.json',
 'wordCloud.ipynb',
 'TopicModeling.ipynb',
 'TopicModelingLDA.ipynb',
 'corpus.pkl',
 'dictionary.gensim',
 'model30.gensim.state',
 'model30.gensim.id2word',
 'model30.gensim.expElogbeta.npy',
 'model30.gensim']

In [35]:
# Load data
# Load data
with open ('preprocessed_tweets_no_dups.pickle', 'rb') as json_file:
    tweets = pickle.load(json_file)

In [121]:

# Load data
infile = open('tweets_topics_LDA.json','rb')
check = json.load(infile)
infile.close()

In [122]:
c = 0
for t in check:
  if t['topic'] == 'Unclassified':
    c+=1

In [123]:
c

3349

In [119]:
len(tweets)

37754

### Fit models

#### LDA (Latent Dirichlet Allocation)
Code adapted from https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/topic_modeling_Gensim.ipynb

In [36]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
'''   
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
'''

'   \nfrom nltk.stem.wordnet import WordNetLemmatizer\ndef get_lemma2(word):\n    return WordNetLemmatizer().lemmatize(word)\n'

In [37]:
def getWords(docs):
    words = []
    for t in tweets:
        # tokenize
        t = re.split('\s',t)
        #for w in t:
        #  lemma = (get_lemma(w))        
        words.append(t)
    return words

words = getWords(tweets)

<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-37-7fdfc2340347>:5: DeprecationWarning: invalid escape sequence \s
  t = re.split('\s',t)


In [38]:
def prepare_text_for_lda(text):
    tokens = text
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [43]:
import random
text_data = []
for line in words:
    tokens = prepare_text_for_lda(line)
    #if random.random() > .99:
      #print(tokens)
    text_data.append(tokens)

In [44]:
len(text_data)

37754

In [45]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [46]:
# Convert document into the bag-of-words format = list of (token_id, token_count)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [47]:
import pickle
pickle.dump(corpus, open('corpus_no_dups.pkl', 'wb'))
dictionary.save('dictionary_no_dups.gensim')

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


LDA Parameters:
- num_topics: number of requested latent topics to be extracted from the training corpus.
- id2word: apping from word IDs to words. It is used to determine the vocabulary size, as well as for debugging and topic printing.
- passes: number of passes through the corpus during training.

In [48]:
import gensim
NUM_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model30.gensim')

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [49]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)
# print_topics(num_topics=20, num_words=10) # To select num of topics to print
# print_topic(topicno, topn=10) # To print one topic

(17, '0.063*"without" + 0.038*"center" + 0.030*"government" + 0.029*"breaking" + 0.026*"school"')
(26, '0.117*"hospital" + 0.043*"thousand" + 0.040*"covid" + 0.040*"patient" + 0.028*"morning"')
(13, '0.048*"clear" + 0.047*"crown" + 0.029*"wearamask" + 0.024*"completely" + 0.017*"train"')
(0, '0.048*"health" + 0.041*"disinfect" + 0.027*"country" + 0.026*"always" + 0.020*"passenger"')
(8, '0.051*"arrive" + 0.033*"important" + 0.028*"tapabocas" + 0.027*"campaign" + 0.026*"happy"')
(25, '0.049*"world" + 0.027*"leave" + 0.026*"people" + 0.024*"party" + 0.019*"visit"')
(1, '0.107*"vaccine" + 0.074*"covid" + 0.053*"death" + 0.032*"vaccination" + 0.019*"population"')
(24, '0.029*"serve" + 0.024*"seem" + 0.023*"quedateencasasa" + 0.023*"given" + 0.018*"saying"')
(11, '0.045*"total" + 0.036*"follow" + 0.034*"corona" + 0.031*"india" + 0.027*"passport"')
(19, '0.113*"symptom" + 0.050*"ómicron" + 0.042*"maintain" + 0.026*"problem" + 0.019*"francisco"')
(20, '0.067*"coronavirus" + 0.064*"detect" + 0

In [78]:
from pprint import pprint
top_topics = list(ldamodel.top_topics(corpus))
pprint(top_topics)
#len(top_topics)

[([(0.07050124, 'covid'),
   (0.04446589, 'vaccine'),
   (0.034357145, 'vaccinate'),
   (0.023723733, 'distance'),
   (0.02303563, 'social'),
   (0.018626183, 'people'),
   (0.016727863, 'omicron'),
   (0.015286271, 'mask'),
   (0.014447979, 'transmission'),
   (0.011716419, 'vaccination'),
   (0.009036495, 'without'),
   (0.0070562684, 'variant'),
   (0.006737206, 'already'),
   (0.006489155, 'crown'),
   (0.006483488, 'case'),
   (0.006369887, 'still'),
   (0.006230151, 'years'),
   (0.006197041, 'death'),
   (0.0061932052, 'prevent'),
   (0.0061301896, 'month')],
  -3.5095978577000726),
 ([(0.047641207, 'covid'),
   (0.034761727, 'variant'),
   (0.034456145, 'omicron'),
   (0.029754495, 'case'),
   (0.022671679, 'first'),
   (0.020708686, 'state'),
   (0.016847076, 'confirm'),
   (0.0150364, 'report'),
   (0.0147811705, 'africa'),
   (0.014661652, 'south'),
   (0.013803947, 'unite'),
   (0.013116105, 'death'),
   (0.012739519, 'country'),
   (0.012271026, 'measure'),
   (0.009445909

In [26]:
#get_document_topics(corpus, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)
t = ldamodel.get_document_topics(corpus)

In [51]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 3),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1)],
 [(19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 2)],
 [(11, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(43, 1), (44, 3), (45, 1), (46, 1), (47, 1), (48, 2)],
 [(49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1)],
 [(2, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1)],
 [(75, 1), (76, 1), (77, 1), (78, 1)],
 [(2, 1),
  (62, 1),
  (76, 1),
  (77, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1)

In [ ]:
'''
# Infer topic distribution on new, unseen documents
new_doc = 'New Covid cases in the U.S. have increased by 25% in the past two weeks. In 14 states, cases have climbed by 40% or more.'
#new_doc = preprocess(new_doc,'en')
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))
'''

[(44, 1), (140, 2), (181, 1), (341, 1), (883, 1)]
[(0, 0.19992442), (1, 0.028729465), (2, 0.028696269), (3, 0.308239), (4, 0.43441084)]


In [64]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

(0, '0.024*"stayhome" + 0.022*"california" + 0.022*"pressure" + 0.021*"hands" + 0.020*"weathercloud"')
(1, '0.039*"contagion" + 0.020*"humidity" + 0.012*"washing" + 0.011*"going" + 0.010*"avoid"')
(2, '0.029*"hospital" + 0.022*"health" + 0.009*"happen" + 0.009*"order" + 0.008*"really"')
(3, '0.048*"covid" + 0.035*"variant" + 0.034*"omicron" + 0.030*"case" + 0.023*"first"')
(4, '0.023*"quedateencasa" + 0.019*"cover" + 0.014*"chile" + 0.012*"santiago" + 0.011*"laflorida"')
(5, '0.046*"virus" + 0.031*"variant" + 0.024*"doctor" + 0.024*"disease" + 0.019*"coronavirus"')
(6, '0.020*"immunity" + 0.015*"epidemic" + 0.013*"today" + 0.010*"people" + 0.007*"press"')
(7, '0.071*"covid" + 0.044*"vaccine" + 0.034*"vaccinate" + 0.024*"distance" + 0.023*"social"')
(8, '0.059*"covid" + 0.034*"positive" + 0.033*"symptom" + 0.018*"patient" + 0.016*"arrive"')
(9, '0.023*"temperature" + 0.020*"disinfect" + 0.016*"distance" + 0.014*"social" + 0.013*"maintain"')


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


#### Visualization with pyLDAvis

In [68]:
dictionary = gensim.corpora.Dictionary.load('dictionary_no_dups.gensim')
corpus = pickle.load(open('corpus_no_dups.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model10.gensim')

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is

In [69]:
import pyLDAvis
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [70]:
# Save plot as html file
pyLDAvis.save_html(lda_display, 'lda_10Topics.html')

In [71]:
def format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=tweets):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [72]:
import pandas as pd
x =  format_topics_sentences()

In [60]:
# To display the entire table and be able to inspect topics
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [62]:
#%load_ext google.colab.data_table

In [73]:
x.head(50)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,6.0,0.4396,"immunity, epidemic, today, people, press, conf...",light menorah white house jewish families plac...
1,2.0,0.3744,"hospital, health, happen, order, really, witho...",build back better act one earning less pay sin...
2,3.0,0.3669,"covid, variant, omicron, case, first, state, c...",tune first lady vice president second gentlema...
3,7.0,0.9100,"covid, vaccine, vaccinate, distance, social, p...",got covid vaccine june time get booster shot a...
4,1.0,0.2912,"contagion, humidity, washing, going, avoid, so...",port action plan already easing bottlenecks go...
5,1.0,0.5506,"contagion, humidity, washing, going, avoid, so...",one republican senator said rising prices “a g...
6,6.0,0.8200,"immunity, epidemic, today, people, press, conf...",tune deliver remarks commemorate world aids day
7,6.0,0.4419,"immunity, epidemic, today, people, press, conf...",tune deliver remarks work strengthen nation su...
8,7.0,0.8714,"covid, vaccine, vaccinate, distance, social, p...",vaccinated still worried omicron go get booste...
9,6.0,0.6364,"immunity, epidemic, today, people, press, conf...",every american family deserves affordable chil...


In [136]:
# Save data frame to csv file
x.to_csv('TopicsPerTweet.csv')

In [126]:
,# Create dictionary with topics names and topics
dic10 = {
    'Stay Home': [0],
    'Contagion': [1],
    'Hospital and health': [2],
    'Covid and variants': [3],
    'Quarantine': [4],
    'Disease': [5],
    'Epidemic and immunity': [6],
    'Vaccination': [7],
    'Positive cases and Symptoms': [8],
    'Social distance': [9]
}

In [127]:
# Get topic labels
def getTopics():

  all_topics = []

  # Get Dominant topic from df
  col_top_list = x['Dominant_Topic'].tolist()

  # Get name for dominant topic
  for item in col_top_list:    
    for key in dic10.keys():
      # If yes, topic = key
      #if str(int(item)) == key:
      #  all_topics.append(dic10[key])
      if int(item) in dic10[key]:
        all_topics.append(key)
  
  return all_topics


In [128]:
all_topics = getTopics() 

In [129]:
len(all_topics)

37754

In [134]:
# Update json file with topics
def updateJsonFile():
    jsonFile = open("tweets_no_dups.json", "r") 
    data = json.load(jsonFile) 
    jsonFile.close()
    
    all_topics = getTopics() #
    i = 0
    for tweet in tqdm(data): 
      tweet['topic'] = all_topics[i]
      i += 1

    with open("tweets_topics_LDA.json", "w+") as jsonFile: 
        jsonFile.write(json.dumps(data))

In [135]:
import json
from tqdm import tqdm
updateJsonFile()

100%|██████████| 37754/37754 [00:00<00:00, 1182894.74it/s]
